In [2]:
import ner
import os
path_model = '../../../ML_EntityData/output/en/model'

In [3]:
model = ner.restore_model_trained(model_folder = path_model,
                                        embedding_filepath='../../../ML_EntityData/embedding/en/glove.6B.100d.txt')

embedded_characters: Tensor("character_embedding/embedded_characters:0", shape=(?, ?, 25), dtype=float32)
embedded_tokens: Tensor("token_embedding/embedding_lookup:0", shape=(?, 100), dtype=float32)
token_lstm_input: Tensor("concatenate_token_and_character_vectors/token_lstm_input:0", shape=(?, 163), dtype=float32)
token_lstm_input_drop: Tensor("dropout/token_lstm_input_drop/mul:0", shape=(?, 163), dtype=float32)
token_lstm_input_drop_expanded: Tensor("dropout/token_lstm_input_drop_expanded:0", shape=(1, ?, 163), dtype=float32)
unary_scores_expanded: Tensor("crf/unary_scores_expanded:0", shape=(1, ?, 19), dtype=float32)
input_label_indices_flat_batch: Tensor("crf/input_label_indices_flat_batch:0", shape=(1, ?), dtype=int32)
sequence_lengths: Tensor("crf/sequence_lengths:0", shape=(1,), dtype=int32)


C:\Users\linhpn.VISC\AppData\Local\Continuum\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Load token embeddings... done (0.17 seconds)
number_of_token_original_case_found: 14618
number_of_token_lowercase_found: 11723
number_of_token_digits_replaced_with_zeros_found: 119
number_of_token_lowercase_and_digits_replaced_with_zeros_found: 16
number_of_loaded_word_vectors: 26476
dataset.vocabulary_size: 28984
INFO:tensorflow:Restoring parameters from ../../../ML_EntityData/output/en/model\model.ckpt


In [2]:
parameters = {#'pretrained_model_folder':'../model', 
                      'dataset_text_folder':'../../../ML_EntityData/data/en', 
                      'character_embedding_dimension':25, 
                      'character_lstm_hidden_state_dimension':25, 
                      'check_for_digits_replaced_with_zeros':True, 
                      'check_for_lowercase':True, 
#                       'debug':False,
#                       'dropout_rate':0.5,
#                       'experiment_name':'test',
                      'freeze_token_embeddings':False,
                      'gradient_clipping_value':5.0,
                      'learning_rate':0.005,
#                       'load_only_pretrained_token_embeddings':False,
#                       'load_all_pretrained_token_embeddings':False,
#                       'main_evaluation_mode':'conll',
                      'maximum_number_of_epochs':500,
#                       'number_of_cpu_threads':8,
#                       'number_of_gpus':0,
                      'optimizer':'sgd',
#                       'output_folder':'../../../ML_EntityData/output',
#                       'patience':10,
#                       'plot_format':'pdf',
#                       'reload_character_embeddings':True,
#                       'reload_character_lstm':True,
#                       'reload_crf':True,
#                       'reload_feedforward':True,
#                       'reload_token_embeddings':True,
#                       'reload_token_lstm':True,
#                       'remap_unknown_tokens_to_unk':True,
#                       'spacylanguage':'en',
#                       'tagging_format':'bioes',
                      'token_embedding_dimension':100,
                      'token_lstm_hidden_state_dimension':100,
                      'token_pretrained_embedding_filepath':'../../../ML_EntityData/embedding/en/glove.6B.100d.txt',
#                       'tokenizer':'spacy',
#                       'train_model':True,
#                       'use_character_lstm':True,
#                       'use_crf':True,
#                       'use_pretrained_model':False,
                      'verbose':False}

In [3]:
import utils_nlp
import pickle
# dataset_filepaths, dataset_brat_folders = utils.get_valid_dataset_filepaths(parameters)
# dataset = ds.DatasetP(verbose=False, debug=False)
token_to_vector = utils_nlp.load_pretrained_token_embeddings(parameters['token_pretrained_embedding_filepath'])
dataset = pickle.load(open('../../../ML_EntityData/output/en/model/dataset.pickle', 'rb'))

In [4]:
model = ner.NER(parameters,dataset = dataset)

embedded_characters: Tensor("character_embedding/embedded_characters:0", shape=(?, ?, 25), dtype=float32)
embedded_tokens: Tensor("token_embedding/embedding_lookup:0", shape=(?, 100), dtype=float32)
token_lstm_input: Tensor("concatenate_token_and_character_vectors/token_lstm_input:0", shape=(?, 163), dtype=float32)
token_lstm_input_drop: Tensor("dropout/token_lstm_input_drop/mul:0", shape=(?, 163), dtype=float32)
token_lstm_input_drop_expanded: Tensor("dropout/token_lstm_input_drop_expanded:0", shape=(1, ?, 163), dtype=float32)
unary_scores_expanded: Tensor("crf/unary_scores_expanded:0", shape=(1, ?, 19), dtype=float32)
input_label_indices_flat_batch: Tensor("crf/input_label_indices_flat_batch:0", shape=(1, ?), dtype=int32)
sequence_lengths: Tensor("crf/sequence_lengths:0", shape=(1,), dtype=int32)


C:\Users\linhpn.VISC\AppData\Local\Continuum\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Load token embeddings... done (35.45 seconds)
number_of_token_original_case_found: 14618
number_of_token_lowercase_found: 11723
number_of_token_digits_replaced_with_zeros_found: 119
number_of_token_lowercase_and_digits_replaced_with_zeros_found: 16
number_of_loaded_word_vectors: 26476
dataset.vocabulary_size: 28984


In [6]:
ner.restore_model_trained(model_pathfile=('../model/en/model.ckpt'),
                                        dataset_pathfile=('../model/en/dataset.pickle'),
                                        embedding_filepath= parameters['token_pretrained_embedding_filepath'],
                                        character_dimension = parameters['character_embedding_dimension'],
                                        token_dimension=parameters['token_embedding_dimension'])

INFO:tensorflow:Restoring parameters from ../model/en/model.ckpt


In [13]:
model.train(3,'../../../ML_EntityData/output/en/model')


Starting epoch 0
Training completed in 0.00 seconds
Evaluate model on the train set
             precision    recall  f1-score   support

      B-LOC     0.8756    0.9643    0.9178      7140
     B-MISC     0.6608    0.9799    0.7894      3438
      B-ORG     0.8335    0.8783    0.8553      6321
      B-PER     0.8509    0.9618    0.9030      6600
      I-LOC     0.8414    0.7839    0.8116      1157
     I-MISC     0.8190    0.7169    0.7645      1155
      I-ORG     0.9979    0.3907    0.5615      3704
      I-PER     0.9924    0.7747    0.8701      4528
          O     0.0000    0.0000    0.0000       712

avg / total     0.8493    0.8299    0.8193     34755

Evaluate model on the valid set
             precision    recall  f1-score   support

      B-LOC     0.8695    0.9575    0.9114      1837
     B-MISC     0.6533    0.9382    0.7703       922
      B-ORG     0.8143    0.8337    0.8239      1341
      B-PER     0.8121    0.9571    0.8786      1842
      I-LOC     0.8297    0.739


Evaluate model on the valid set
             precision    recall  f1-score   support

      B-LOC     0.9072    0.9260    0.9165      1837
     B-MISC     0.6769    0.9111    0.7767       922
      B-ORG     0.8081    0.8479    0.8275      1341
      B-PER     0.8446    0.9620    0.8995      1842
      I-LOC     0.9627    0.6031    0.7416       257
     I-MISC     0.7821    0.5809    0.6667       346
      I-ORG     0.9908    0.2863    0.4442       751
      I-PER     0.9794    0.7995    0.8804      1307
          O     0.0000    0.0000    0.0000       113

avg / total     0.8573    0.8107    0.8114      8716

Evaluate model on the test set
             precision    recall  f1-score   support

      B-LOC     0.8450    0.8693    0.8570      1668
     B-MISC     0.5383    0.8718    0.6656       702
      B-ORG     0.7745    0.8170    0.7952      1661
      B-PER     0.8367    0.9505    0.8900      1617
      I-LOC     0.8138    0.4591    0.5871       257
     I-MISC     0.6308    0.569

In [5]:
_ , entitys = model.quick_predict('my name is Jon Jgo. ')
entitys

[('Jon Jgo', 'PER')]

In [11]:
import codecs
text_filepath = '../data/test/cv.txt'
with codecs.open(text_filepath, 'r', 'UTF-8') as f:
    text = f.read()
a,entitys = model.quick_predict(text)
entitys,a

([('Le', 'ORG'),
  ('Sex', 'PER'),
  ('Address', 'MISC'),
  ('Thai', 'MISC'),
  ('Street', 'LOC'),
  ('Ba Dinh', 'PER'),
  ('Hanoi', 'LOC'),
  ('Developer', 'MISC'),
  ('iOS', 'LOC'),
  ('watchOS', 'MISC'),
  ('OS', 'MISC'),
  ('&', 'MISC'),
  ('Swift', 'MISC'),
  ('Developer', 'MISC'),
  ('Emotiv', 'LOC'),
  ('Silicon', 'MISC'),
  ('Valley', 'LOC'),
  ('iOS', 'MISC'),
  ('Insights ”', 'MISC'),
  ('Metal Commands', 'MISC'),
  ('iOS', 'MISC'),
  ('SDK', 'ORG'),
  ('Swift', 'MISC'),
  ('Insights', 'MISC'),
  ('SDK', 'MISC'),
  ('Software', 'MISC'),
  ('Vietnam', 'LOC'),
  ('Production', 'MISC'),
  ('Core', 'MISC'),
  ('Core', 'MISC'),
  ('iOS', 'MISC'),
  ('Swift', 'MISC'),
  ('iOS', 'MISC'),
  ('Event Organizer', 'PER'),
  ('iOS', 'MISC'),
  ('Techmaster', 'MISC'),
  ('iOS', 'MISC'),
  ('Sweepers', 'MISC'),
  ('Snake', 'MISC'),
  ('Developer', 'PER'),
  ('iOS', 'MISC'),
  ('iOS', 'MISC'),
  ('Techmaster', 'MISC'),
  ('Vietnam', 'LOC'),
  ('Vietnam', 'LOC'),
  ('Trinh Minh Cuong', 'PER')

In [8]:
import sklearn.metrics as metrics


In [9]:
y1=[1,2,3,4,2,3,4]
y2=[2,1,3,3,2,3,4]

In [15]:
[float(t) for t in [t for t in metrics.classification_report(y1,y2).split('\n')[-2].split(' ') if len(t)>0][-4:]]

[0.62, 0.57, 0.56, 7.0]

In [12]:
float('1')

1.0